# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import os
import requests
import sqlite3
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score,confusion_matrix,precision_score, recall_score, f1_score, classification_report, fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import average_precision_score,roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DB_Disasters_clean', engine)
X = df['message']
Y = df[df.columns[4:]]

In [3]:
Y.head(5)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X.head(5)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
look=Y.iloc[:, 4:].isnull().sum()/df.shape[0]
print(look.head(10))

medical_help         0.0
medical_products     0.0
search_and_rescue    0.0
security             0.0
military             0.0
child_alone          0.0
water                0.0
food                 0.0
shelter              0.0
clothing             0.0
dtype: float64


In [6]:
look2=Y.iloc[:, 4:].sum()/df.shape[0]
print(look2.sort_values().head(10))

child_alone             0.000000
medical_help                 inf
cold                         inf
earthquake                   inf
fire                         inf
storm                        inf
floods                       inf
weather_related              inf
other_infrastructure         inf
aid_centers                  inf
dtype: float64


In [7]:
Y = Y.drop(['child_alone'], axis = 1)

In [8]:
col_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [10]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'only', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
        ('features', FeatureUnion([
              ('text_pipeline', Pipeline([
                   ('vect', CountVectorizer(tokenizer=tokenize)),
                   ('tfidf', TfidfTransformer()),
              ])),
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [13]:
pip_fit = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pd.DataFrame(pip_fit.predict(X_test))
y_pred_t = pip_fit.predict(X_test)
print(y_pred.head(5))

  0  1  2  3  4  5  6  7  8  9  ... 25 26 27 28 29 30 31 32 33 34
0  1  0  0  1  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
1  0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
2  1  0  0  1  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
3  1  0  0  1  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
4  1  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0

[5 rows x 35 columns]


In [15]:
#print(classification_report(y_test, y_pred, target_names=Y.columns.values))
for i, c in enumerate(col_names):

        print(c)

        print(classification_report(y_test.iloc[:,i],y_pred.iloc[:,i]))

related
             precision    recall  f1-score   support

          0       0.62      0.47      0.53      1535
          1       0.85      0.91      0.88      5010

avg / total       0.80      0.81      0.80      6545

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5396
          1       0.81      0.42      0.55      1149

avg / total       0.87      0.88      0.86      6545

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6513
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6545

aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3811
          1       0.76      0.60      0.67      2734

avg / total       0.75      0.75      0.75      6545

medical_help


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.93      0.99      0.96      6036
          1       0.50      0.06      0.11       509

avg / total       0.89      0.92      0.89      6545

medical_products
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6186
          1       0.74      0.12      0.21       359

avg / total       0.94      0.95      0.93      6545

search_and_rescue
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      6344
          1       0.59      0.05      0.09       201

avg / total       0.96      0.97      0.96      6545

security
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      6422
          1       0.20      0.01      0.02       123

avg / total       0.97      0.98      0.97      6545

military
             precision    recall  f1-score   support

          0       0.97      1.00 

In [16]:
pr=[]
ac=[]

for i in range(0,len(col_names),1):
    pr.append(precision_score(y_test.iloc[:, i],y_pred.iloc[:, i],average='macro'))
    ac.append(accuracy_score(y_test.iloc[:, i], y_pred.iloc[:, i]))
    
ac=pd.DataFrame(ac)
pr=pd.DataFrame(pr)

print('Mean Precision',pr.mean())
print('Mean Accuracy',ac.mean())

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Mean Precision 0    0.739638
dtype: float64
Mean Accuracy 0    0.942744
dtype: float64


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline2 = Pipeline([
            ('features', FeatureUnion([
                  ('text_pipeline', Pipeline([
                                               ('vect', CountVectorizer(tokenizer=tokenize)),
                                               ('tfidf', TfidfTransformer()),
                                             ])
                  ),
            ])),
              ("classifier", MultiOutputClassifier(RandomForestClassifier()))
           ])
parameters = {
            'classifier__estimator__max_depth': [ 8, 16, 32],
            'classifier__estimator__n_estimators': [50, 100, 150]
}

cv = GridSearchCV(pipeline2, param_grid=parameters)

In [ ]:
model = cv.fit(X_train,y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_improve = pd.DataFrame(model.predict(X_test))

In [ ]:
#print(classification_report(y_test, y_pred, target_names=Y.columns.values))
for i, c in enumerate(col_names):

        print(c)

        print(classification_report(y_test.iloc[:,i],y_pred_improve.iloc[:,i]))

In [ ]:
pr=[]
ac=[]
rc=[]
for i in range(0,len(col_names),1):
    pr.append(precision_score(y_test.iloc[:, i],y_pred_improve.iloc[:, i],average='macro'))
    rc.append(recall_score(y_test.iloc[:, i],y_pred_improve.iloc[:, i],average='macro'))
    ac.append(accuracy_score(y_test.iloc[:, i], y_pred_improve.iloc[:, i]))

    
ac=pd.DataFrame(ac)
pr=pd.DataFrame(pr)
rc=pd.DataFrame(rc)

print('Mean Precision',pr.mean())
print('Mean Accuracy',ac.mean())
print('Mean Recall',rc.mean())

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

0.8818787781602917

### 9. Export your model as a pickle file

In [24]:
model_out = pickle.dumps('classifier.pkl')
#dump(xgb_model,open('clf.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.